In [13]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import gensim
import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")

tweet_clean=pd.read_csv('/Users/kejinglin/Desktop/personal/mydata/tweet_clean.csv',encoding='utf-8')
tweet_clean.head()

,u_nickname,u_url,m_content,m_content_url,m_content_id,m_images,m_videos,r_comment_num,g_publish_time,r_like_num,r_trans_num,verified,u_id,u_desc,u_fans,u_follow,u_login_time,u_area
0,Senator John Cornyn,https://twitter.com/JohnCornyn,"""A University of Southampton study suggests th...",NaN,1247121007749607429,NaN,NaN,231,2020-04-06 19:18:47,509,218,1,13218102.0,"Mainly news I find interesting, pics, and opin...",294161,11996,2008-02-08 03:52:55,"Austin, Texas"
1,Paul Joseph Watson,https://twitter.com/PrisonPlanet,"""A study published in March indicated that if ...",NaN,1240429794057236483,NaN,NaN,224,2020-03-19 08:08:03,11594,3385,1,18643437.0,"""Watson is a brilliant polemicist."" The Specta...",1108941,770,2009-01-06 04:04:23,London
2,Brian Stelter,https://twitter.com/brianstelter,"""America has had the best response to coronavi...",NaN,1239359123587846144,NaN,NaN,938,2020-03-16 09:13:35,1455,184,1,14515799.0,Anchor of @ReliableSources and @CNN's chief me...,781737,7075,2008-04-25 02:41:42,New York City
3,Laurie Garrett,https://twitter.com/Laurie_Garrett,"""As many as 14% of recovered #coronavirus pati...",NaN,1243328270059405312,NaN,NaN,127,2020-03-27 08:05:34,970,829,1,299273962.0,Former Sr Fellow @CFR_org. Recipient of Pulitz...,237918,1818,2011-05-16 04:00:40,"New York, NY"
4,Tom Fitton,https://twitter.com/TomFitton,"""BIG GUY"" Biden implicated in RICO-style deal ...",NaN,1319789796185772033,['http://pbs.twimg.com/media/ElDVv-aWkAAE1d3.j...,NaN,435,2020-10-24 07:56:22,3789,1580,1,18266688.0,"President, Judicial Watch. (These are my perso...",1184751,3225,2008-12-20 22:32:44,"Washington, DC"


In [14]:
corpus=[]
a=[]
for i in range(len(tweet_clean['m_content'])):
        a=tweet_clean['m_content'][i]
        corpus.append(a)
        
corpus[0:10]  

['"A University of Southampton study suggests the number of coronavirus cases could have been reduced by 95% had China moved to contain the virus three weeks sooner."',
 '"A study published in March indicated that if Chinese authorities had acted three weeks earlier than they did, the number of coronavirus cases could have been reduced by 95% and its geographic spread limited."\n\nMedia: Boo hoo, stop being mean to China!',
 '"America has had the best response to coronavirus in the world!"\n\n—@mattgaetz on Fox',
 '"As many as 14% of recovered #coronavirus patients in China have tested positive again...betwn 3 to 14% of patients were diagnosed w/ the coronavirus, after already being given the all-clear."\nA 2nd wave coming?\nAre #COVID19 tests failing?\nSuperinfection? https://t.co/Jg3SsL94lc',
 '"BIG GUY" Biden implicated in RICO-style deal w/Chinese communists! Should @RealDonaldTrump trust FBI to investigate? Fauci emails show he approved WHO praise of China on #coronavirus transpar

In [16]:
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "/var/folders/g8/fjfkltz11ml925dqpwv1hq740000gn/T" will be used to save temporary dictionary and corpus.


In [17]:
# removing common words and tokenizing
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference

#print(dictionary)
#print(dictionary.token2id)


2021-02-19 13:54:51,330 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-19 13:54:51,446 : INFO : built Dictionary(18868 unique tokens: ['"a', '95%', 'cases', 'china', 'contain']...) from 3363 documents (total 70162 corpus positions)
2021-02-19 13:54:51,447 : INFO : saving Dictionary object under /var/folders/g8/fjfkltz11ml925dqpwv1hq740000gn/T/elon.dict, separately None
2021-02-19 13:54:51,455 : INFO : saved /var/folders/g8/fjfkltz11ml925dqpwv1hq740000gn/T/elon.dict


In [18]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  

2021-02-19 13:55:09,537 : INFO : storing corpus in Matrix Market format to /var/folders/g8/fjfkltz11ml925dqpwv1hq740000gn/T/elon.mm
2021-02-19 13:55:09,540 : INFO : saving sparse matrix to /var/folders/g8/fjfkltz11ml925dqpwv1hq740000gn/T/elon.mm
2021-02-19 13:55:09,541 : INFO : PROGRESS: saving document #0
2021-02-19 13:55:09,571 : INFO : PROGRESS: saving document #1000
2021-02-19 13:55:09,602 : INFO : PROGRESS: saving document #2000
2021-02-19 13:55:09,633 : INFO : PROGRESS: saving document #3000
2021-02-19 13:55:09,647 : INFO : saved 3363x18868 matrix, density=0.106% (67298/63453084)
2021-02-19 13:55:09,648 : INFO : saving MmCorpus index to /var/folders/g8/fjfkltz11ml925dqpwv1hq740000gn/T/elon.mm.index


In [20]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2021-02-19 13:55:35,330 : INFO : collecting document frequencies
2021-02-19 13:55:35,330 : INFO : PROGRESS: processing document #0
2021-02-19 13:55:35,346 : INFO : calculating IDF weights for 3363 documents and 18868 features (67298 matrix non-zeros)


In [21]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

In [22]:
total_topics = 7
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow

2021-02-19 13:56:22,275 : INFO : using symmetric alpha at 0.14285714285714285
2021-02-19 13:56:22,276 : INFO : using symmetric eta at 0.14285714285714285
2021-02-19 13:56:22,280 : INFO : using serial LDA version on this node
2021-02-19 13:56:22,296 : INFO : running online (single-pass) LDA training, 7 topics, 1 passes over the supplied corpus of 3363 documents, updating model once every 2000 documents, evaluating perplexity every 3363 documents, iterating 50x with a convergence threshold of 0.001000
2021-02-19 13:56:22,297 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-02-19 13:56:22,298 : INFO : PROGRESS: pass 0, at document #2000/3363
2021-02-19 13:56:23,723 : INFO : merging changes from 2000 documents into a model of 3363 documents
2021-02-19 13:56:23,736 : INFO : topic #5 (0.143): 0.027*"china" + 0.011*"coronavirus" + 0.011*"virus" + 0.009*"&amp;" + 0.008*"chinese" + 0.007*"world" + 0.006*"co

In [24]:
#Show first n important word in the topics:
lda.show_topics(total_topics,20)

[(0,
  '0.008*"china" + 0.006*"covid" + 0.004*"covid-19" + 0.003*"&amp;" + 0.003*"virus" + 0.003*"global" + 0.002*"china." + 0.002*"deaths" + 0.002*"us" + 0.002*"spread" + 0.002*"china," + 0.002*"account" + 0.002*"#covid19" + 0.002*"new" + 0.002*"government" + 0.002*"lockdown" + 0.002*"take" + 0.002*"relief" + 0.002*"even" + 0.001*"trump"'),
 (1,
  '0.027*"china" + 0.012*"&amp;" + 0.009*"coronavirus" + 0.008*"trump" + 0.007*"chinese" + 0.006*"people" + 0.006*"virus" + 0.005*"must" + 0.005*"china." + 0.004*"#coronavirus" + 0.004*"medical" + 0.003*"us" + 0.003*"world" + 0.003*"blame" + 0.003*"americans" + 0.003*"#china" + 0.003*"pandemic" + 0.003*"communist" + 0.003*"china’s" + 0.003*"virus."'),
 (2,
  '0.025*"china" + 0.009*"coronavirus" + 0.009*"virus" + 0.008*"trump" + 0.007*"&amp;" + 0.007*"us" + 0.005*"china." + 0.005*"china," + 0.005*"covid-19" + 0.004*"president" + 0.003*"one" + 0.003*"u.s." + 0.003*"travel" + 0.003*"get" + 0.002*"crisis" + 0.002*"said" + 0.002*"coronavirus." + 0.

In [25]:
data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [26]:
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(7, 74)


In [27]:
df_lda

,china,covid,covid-19,&amp;,virus,global,china.,deaths,us,spread,...,first,italy,want,flu,confirmed,korea,→,united,virus”,times
0,0.007559,0.006396,0.003784,0.003286,0.003135,0.002594,0.002395,0.002352,0.002316,0.002099,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.026668,0.002415,0.002586,0.011895,0.005605,0.000000,0.004575,0.000000,0.003441,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.024568,0.000000,0.004674,0.006991,0.008948,0.000000,0.005022,0.000000,0.006910,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.028526,0.000000,0.003925,0.005234,0.008984,0.000000,0.004403,0.000000,0.005884,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,0.034541,0.005863,0.000000,0.003516,0.005428,0.002565,0.000000,0.002166,0.006891,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
5,0.029621,0.000000,0.006354,0.011036,0.012520,0.000000,0.006973,0.000000,0.002839,0.006033,...,0.003156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,0.013581,0.003825,0.003486,0.000000,0.002622,0.000000,0.003183,0.000000,0.002754,0.000000,...,0.002235,0.003428,0.002599,0.002383,0.002128,0.002124,0.002101,0.002038,0.002024,0.00201


In [60]:
import nltk
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer  
import xlrd


if __name__ == "__main__":
    corpus = all_words
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    word = vectorizer.get_feature_names()   # 所有的特征词，即关键词
#     print(word)    
    analyze = vectorizer.build_analyzer()  
    weight = X.toarray()  
#     print(weight)
    
    import numpy as np
    import lda
    
    # 训练模型
    model = lda.LDA(n_topics = 7, n_iter = 500, random_state = 1)
    model.fit(np.asarray(weight))
    
    # 主题-词分布
    topic_word = model.topic_word_  #生成主题以及主题中词的分布
    #print("topic-word:\n", topic_word)
    
    # 计算topN关键词
    n = 20  
    for i, word_weight in enumerate(topic_word):  
        #print("word_weight:\n", word_weight)
        distIndexArr = np.argsort(word_weight)
        #print("distIndexArr:\n", distIndexArr)
        topN_index = distIndexArr[:-(n+1):-1]
        #print("topN_index:\n", topN_index) # 权重最在的n个
        topN_words = np.array(word)[topN_index]    
        print(u'*Topic {}\n- {}'.format(i, ' '.join(topN_words))) 
    
    #绘制主题-词分布图
    import matplotlib.pyplot as plt  
    f, ax= plt.subplots(2, 1, figsize=(6, 6), sharex=True)  
    for i, k in enumerate([0,6]):         #两个主题
        ax[i].stem(topic_word[k,:], linefmt='b-',  
                   markerfmt='bo', basefmt='w-')  
        ax[i].set_xlim(-2,20)  
        ax[i].set_ylim(0, 1)  
        ax[i].set_ylabel("Prob")  
        ax[i].set_title("topic {}".format(k))  
    ax[1].set_xlabel("word")  
    plt.tight_layout()  
    plt.show()


ValueError: Iterable over raw text documents expected, string object received.